### Create companies URLs for Bright Data Scraping

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd

# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 1000)
spark = SparkSession.builder.getOrCreate()

In [0]:
import pandas as pd
import json
from pyspark.sql.functions import col

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Out[2]: True

In [0]:
companies = spark.read.parquet('/linkedin/companies')

## Established Companies

In [0]:

# filter old companies 
old_companies = companies.filter((col('founded') >= '1900') & (col('founded') < '2020'))

# take the maximun between 2004 and foundation year
old_companies = old_companies.orderBy(col('followers').desc())
old_companies = old_companies.withColumn('scraping_time', 
                                         when(col('founded') <= '2004', 'all')
                                         .otherwise(concat(col('founded'), lit('-01-01%202024-03-20'))))

pd_df = old_companies.select(['name', 'followers', 'scraping_time']).toPandas()

In [0]:
#preprocess names 
company_names = pd_df['name'].to_list()

preprocessed_company_names = []
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

company_names_dict = {}

search_name_list = []

for name in company_names:
    real_name = name
    # Remove special characters
    name = re.sub(r'\bInc.\b', '', name).strip()
    name = re.sub(r'\bInc\b', '', name).strip()
    name = re.sub(r'\bCo.\b', '', name).strip()
    name = re.sub(r'[^\w\s.,&()-]', '', name)

    if '-' in name or ',' in name or '(' in name:
      if '-' in name:
        parts = name.split('-')
        name = parts[0].strip()  # Get the part before the hyphen and remove any leading/trailing whitespace

      if ',' in name:
        parts = name.split(',')
        name = parts[0].strip()  # Get the part before the hyphen and remove any leading/trailing whitespace
    
      if '(' in name:
        parts = name.split('(')
        informative_part = parts[-1].strip()  # Get the part before the hyphen and remove any leading/trailing whitespace
        informative_part = informative_part.split(')')[0].strip()
        if informative_part.isupper():
          name = informative_part
        else: 
          name = parts[0].strip()

    company_names_dict[real_name] = name
    search_name_list.append(name)
 
pd_df['search_name'] = search_name_list

filtered_company_names_dict = {key: value for key, value in company_names_dict.items() if len(value.split(' ')) <= 4}

In [0]:
# convert names to google trends urls
pd_df['name_length'] = pd_df['search_name'].apply(lambda x: len(x.split(' ')))
pd_df['url'] = [f"https://trends.google.com/trends/explore?date={x}&q={y}" for x, y in zip(pd_df['scraping_time'], pd_df['search_name'])]
pd_df = pd_df.sort_values(by=['followers', 'name_length'], ascending=[False, True])

# create df for the collector in bright data
pyspark_df = spark.createDataFrame(pd_df[['url', 'name', 'followers']][0:40000])
pyspark_df.select('url', 'followers').display()


url,followers
https://trends.google.com/trends/explore?date=all&q=Forbes,18646055
https://trends.google.com/trends/explore?date=all&q=NASA,6478111
https://trends.google.com/trends/explore?date=2015-01-01%202024-03-20&q=The Kraft Heinz Company,1372380
https://trends.google.com/trends/explore?date=2014-01-01%202024-03-20&q=freeCodeCamp,1305165
https://trends.google.com/trends/explore?date=all&q=Manpower,1153949
https://trends.google.com/trends/explore?date=all&q=Kaiser Permanente,862059
https://trends.google.com/trends/explore?date=2009-01-01%202024-03-20&q=PodCast,750472
https://trends.google.com/trends/explore?date=all&q=Genentech,686205
https://trends.google.com/trends/explore?date=all&q=Starwood Hotels & Resorts Worldwide,496771
https://trends.google.com/trends/explore?date=all&q=Sheraton Hotels & Resorts,486386


## New Companies

In [0]:
# filter new companies 
new_companies = companies.filter((col('founded') >= '2020') & (col('founded') < '2025'))

# take the maximun between 2004 and foundation year
new_companies = new_companies.orderBy(col('followers').desc())
new_companies = new_companies.withColumn('scraping_time', 
                                         when(col('founded') <= '2004', 'all')
                                         .otherwise(concat(col('founded'), lit('-01-01%202024-03-20'))))

pd_df_new = new_companies.select(['name', 'followers', 'scraping_time']).toPandas()


In [0]:
# preprocess names
company_names = pd_df_new['name'].to_list()

preprocessed_company_names = []
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

company_names_dict = {}

search_name_list = []

for name in company_names:
    real_name = name
    # Remove special characters
    name = re.sub(r'\bInc.\b', '', name).strip()
    name = re.sub(r'\bInc\b', '', name).strip()
    name = re.sub(r'\bCo.\b', '', name).strip()
    name = re.sub(r'[^\w\s.,&()-]', '', name)

    if '-' in name or ',' in name or '(' in name:
      if '-' in name:
        parts = name.split('-')
        name = parts[0].strip()  # Get the part before the hyphen and remove any leading/trailing whitespace

      if ',' in name:
        parts = name.split(',')
        name = parts[0].strip()  # Get the part before the hyphen and remove any leading/trailing whitespace
    
      if '(' in name:
        parts = name.split('(')
        informative_part = parts[-1].strip()  # Get the part before the hyphen and remove any leading/trailing whitespace
        informative_part = informative_part.split(')')[0].strip()
        if informative_part.isupper():
          name = informative_part
        else: 
          name = parts[0].strip()

    company_names_dict[real_name] = name
    search_name_list.append(name)
 
pd_df_new['search_name'] = search_name_list

filtered_company_names_dict = {key: value for key, value in company_names_dict.items() if len(value.split(' ')) <= 4}

In [0]:
# convert names to google trends urls
pd_df_new['name_length'] = pd_df_new['search_name'].apply(lambda x: len(x.split(' ')))
pd_df_new['url'] = [f"https://trends.google.com/trends/explore?date={x}&q={y}" for x, y in zip(pd_df_new['scraping_time'], pd_df_new['search_name'])]
pd_df_new = pd_df_new.sort_values(by=['followers', 'name_length'], ascending=[False, True])

# create df for the collector in bright data
pyspark_new_df = spark.createDataFrame(pd_df_new[['url', 'name', 'followers']][0:5000])
pyspark_new_df.select('url').display()

url
https://trends.google.com/trends/explore?date=2021-01-01%202024-03-20&q=Kyndryl
https://trends.google.com/trends/explore?date=2023-01-01%202024-03-20&q=Kenvue
https://trends.google.com/trends/explore?date=2020-01-01%202024-03-20&q=The TOP Saudi Arabia 100 Times LinkedIn Powers for Free Global Charity 102 Business Magazines
https://trends.google.com/trends/explore?date=2021-01-01%202024-03-20&q=MarTech
https://trends.google.com/trends/explore?date=2020-01-01%202024-03-20&q=Cerebral
https://trends.google.com/trends/explore?date=2021-01-01%202024-03-20&q=Black Women Handle Business
https://trends.google.com/trends/explore?date=2022-01-01%202024-03-20&q=Green Jobs Board
https://trends.google.com/trends/explore?date=2020-01-01%202024-03-20&q=The TOP Delaware 100 Times LinkedIn Powers for Free Global Charity 102 Business Magazines
https://trends.google.com/trends/explore?date=2020-01-01%202024-03-20&q=Howmet Aerospace
https://trends.google.com/trends/explore?date=2021-01-01%202024-03-20&q=Recruiting from Scratch
